In [306]:
import requests
import pandas

In [307]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd

import math

# I'm creating a list of each genre and sorting the games by genre rather than going to each game's page individually
# The webscraper goes by each of the genres one by one, and since it's already filtered by that genre, it has that data.
genres = [
    "Action",
    "Action-Adventure",
    "Adventure",
    "Board Game",
    "Education",
    "Fighting",
    "Misc",
    "MMO",
    "Music",
    "Party",
    "Platform",
    "Puzzle",
    "Racing",
    "Role-Playing",
    "Sandbox",
    "Shooter",
    "Simulation",
    "Sports",
    "Strategy",
    "Visual Novel"
]
genre_index=0
results_per_page=1000

title_data =[]
console_data =[]
publisher_data = []
developer_data = []
vgchartz_score_data= []
critic_score_data= []
user_score_data = []
total_shipped_data = []
total_sales_data = []
na_sales_data = []
pal_sales_data = []
japan_sales_data = []
other_sales_data = []
release_data = []
last_update_data = []
genre_data = []
        
# I had to manually include the results for each genre, because I the automatic method I tried to make wasn't reliable
# and kept throwing errors. No idea why
total_results = [8401, 1755, 6166, 26, 34, 2319, 9300, 113, 285, 148, 3943, 3471, 3374, 5609, 20, 5332, 3078, 5565, 3643, 491]


for genre_index in range(0, len(genres)):
    genre_result_url = 'https://www.vgchartz.com/games/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre'+genres[genre_index]+'=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=200&order=Sales&showtotalsales=0&showtotalsales=1&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&shownasales=1&showdeveloper=0&showdeveloper=1&showcriticscore=0&showpalsales=0&showpalsales=1&showreleasedate=0&showreleasedate=1&showuserscore=0&showjapansales=0&showlastupdate=0&showlastupdate=1&showothersales=0&showothersales=1&showshipped=0&showshipped=1'
    
    genre_result_page = requests.get(genre_result_url)
    genre_result_soup = BeautifulSoup(genre_result_page.text, 'html')

    #This is the automatic method I tried to make. Sometimes it worked, sometimes it didn't
    #total_results=int(genre_result_soup.find('th', attrs={"colspan": "3"}).text.split()[1][1:-1].replace(",", ""))
   
    #The total page amount is calculated here
    total_pages=math.ceil(total_results[genre_index]/results_per_page)
    
    #Each page is iterated per genre filter and all the data is simply collected from the results table.
    for page_number in range(1, total_pages):
        data_group=[]
        dev_pub =[]
        url = 'https://www.vgchartz.com/games/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre'+genres[genre_index]+'=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results='+str(results_per_page)+'&order=Sales&showtotalsales=0&showtotalsales=1&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&shownasales=1&showdeveloper=0&showdeveloper=1&showcriticscore=0&showpalsales=0&showpalsales=1&showreleasedate=0&showreleasedate=1&showuserscore=0&showjapansales=0&showlastupdate=0&showlastupdate=1&showothersales=0&showothersales=1&showshipped=0&showshipped=1'+str(page_number)
        page = requests.get(url)

        soup = BeautifulSoup(page.text, 'html')
        script_data=soup.find_all('td', attrs={"style": "font-size:12pt;"})
        for title in script_data:
            title.find('a')
            title_data.append(title.text.replace("Read the review", "").strip())
            genre_data.append(genres[genre_index])
        script_data=soup.find_all('td', attrs={"width": "100"})
        for publisher in script_data:
            #title.find('a', attrs={"style": "color:#e60012"})
            dev_pub.append(publisher.text.strip())
        for i in range(0, len(dev_pub),2):
            publisher_data.append(dev_pub[i])
        for i in range(1, len(dev_pub),2):
            developer_data.append(dev_pub[i])
        script_data=soup.find_all('td', attrs={"align": "center"})
        for dg in script_data:
            data_group.append(dg)
        for i in range(1, len(data_group),12):
            console_data.append(data_group[i-1].find('img').get('alt', ''))
            vgchartz_score_data.append(data_group[i].text.strip())
            critic_score_data.append(data_group[i+1].text.strip())
            user_score_data.append(data_group[i+2].text.strip())
            total_shipped_data.append(data_group[i+3].text.strip())
            total_sales_data.append(data_group[i+4].text.strip())
            na_sales_data.append(data_group[i+5].text.strip())
            pal_sales_data.append(data_group[i+6].text.strip())
            japan_sales_data.append(data_group[i+7].text.strip())
            other_sales_data.append(data_group[i+8].text.strip())
            release_data.append(data_group[i+9].text.strip())
            last_update_data.append(data_group[i+10].text.strip())

        

#Column header
column_header=[
    'Title',
    'Console',
    'Publisher',
    'Developer',
    'VGChartz Score',
    'Critic Score',
    'User Score',
    'Total Shipped',
    'Total Sales',
    'NA Sales',
    'PAL Sales',
    'Japan Sales',
    'Other Sales',
    'Release Date',
    'Last Update',
    'Genre'
]


#Data values
column_values=[
        title_data,
        console_data,
        publisher_data,
        developer_data,
        vgchartz_score_data,
        critic_score_data,
        user_score_data,
        total_shipped_data,
        total_sales_data,
        na_sales_data,
        pal_sales_data,
        japan_sales_data,
        other_sales_data,
        release_data,
        last_update_data,
        genre_data
]
                
pd.set_option('display.max_rows', None)

#Add header to dataframe
df=pd.DataFrame(columns=column_header)

#Add data to data frame
for column, value in zip( column_header, column_values):
    df[column] = value


df.to_csv('vg_data1.csv')

AttributeError: 'NoneType' object has no attribute 'get'

In [308]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math
import time

# URL base
base_url = 'https://www.vgchartz.com/games/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=200&order=Sales&showtotalsales=0&showtotalsales=1&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&shownasales=1&showdeveloper=0&showdeveloper=1&showcriticscore=0&showpalsales=0&showpalsales=1&showreleasedate=0&showreleasedate=1&showuserscore=0&showjapansales=0&showjapansales=1&showlastupdate=0&showlastupdate=1&showothersales=0&showothersales=1&showshipped=0&showshipped=1'

# Initialize lists to store data
title_data = []
console_data = []
publisher_data = []
developer_data = []
vgchartz_score_data = []
critic_score_data = []
user_score_data = []
total_shipped_data = []
total_sales_data = []
na_sales_data = []
pal_sales_data = []
japan_sales_data = []
other_sales_data = []
release_data = []
last_update_data = []

# Scraping function
def scrape_page(page_number):
    url = f'{base_url}&page={page_number}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    titles = soup.find_all('td', attrs={"style": "font-size:12pt;"})
    publishers_developers = soup.find_all('td', attrs={"width": "100"})
    data_rows = soup.find_all('td', attrs={"align": "center"})
    
    dev_pub = [x.text.strip() for x in publishers_developers]
    data_group = [x for x in data_rows]
    
    for title in titles:
        title_data.append(title.text.replace("Read the review", "").strip())
    
    for i in range(0, len(dev_pub), 2):
        publisher_data.append(dev_pub[i])
    for i in range(1, len(dev_pub), 2):
        developer_data.append(dev_pub[i])
    
    for i in range(1, len(data_group), 12):
        console_data.append(data_group[i-1].find('img').get('alt', ''))
        vgchartz_score_data.append(data_group[i].text.strip())
        critic_score_data.append(data_group[i+1].text.strip())
        user_score_data.append(data_group[i+2].text.strip())
        total_shipped_data.append(data_group[i+3].text.strip())
        total_sales_data.append(data_group[i+4].text.strip())
        na_sales_data.append(data_group[i+5].text.strip())
        pal_sales_data.append(data_group[i+6].text.strip())
        japan_sales_data.append(data_group[i+7].text.strip())
        other_sales_data.append(data_group[i+8].text.strip())
        release_data.append(data_group[i+9].text.strip())
        last_update_data.append(data_group[i+10].text.strip())

# Determine the number of pages
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')
total_results = int(soup.find('th', attrs={"colspan": "3"}).text.split()[1].replace(",", ""))
results_per_page = 200
total_pages = math.ceil(total_results / results_per_page)

# Scrape each page
for page_number in range(1, total_pages + 1):
    print(f'Scraping page {page_number} of {total_pages}...')
    scrape_page(page_number)
    time.sleep(1)  # Respectful scraping, pause between requests

# Create DataFrame
df = pd.DataFrame({
    'Title': title_data,
    'Console': console_data,
    'Publisher': publisher_data,
    'Developer': developer_data,
    'VGChartz Score': vgchartz_score_data,
    'Critic Score': critic_score_data,
    'User Score': user_score_data,
    'Total Shipped': total_shipped_data,
    'Total Sales': total_sales_data,
    'NA Sales': na_sales_data,
    'PAL Sales': pal_sales_data,
    'Japan Sales': japan_sales_data,
    'Other Sales': other_sales_data,
    'Release Date': release_data,
    'Last Update': last_update_data
})

# Save DataFrame to CSV
df.to_csv('vg_data1.csv', index=False)

print('Scraping completed and data saved to vg_data1.csv')


ValueError: invalid literal for int() with base 10: '(65577)'

In [311]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math
import time

# URL base
base_url = 'https://www.vgchartz.com/games/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=200&order=Sales&showtotalsales=0&showtotalsales=1&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&shownasales=1&showdeveloper=0&showdeveloper=1&showcriticscore=0&showpalsales=0&showpalsales=1&showreleasedate=0&showreleasedate=1&showuserscore=0&showjapansales=0&showjapansales=1&showlastupdate=0&showlastupdate=1&showothersales=0&showothersales=1&showshipped=0&showshipped=1'

# Initialize lists to store data
title_data = []
console_data = []
publisher_data = []
developer_data = []
vgchartz_score_data = []
critic_score_data = []
user_score_data = []
total_shipped_data = []
total_sales_data = []
na_sales_data = []
pal_sales_data = []
japan_sales_data = []
other_sales_data = []
release_data = []
last_update_data = []

# Scraping function
def scrape_page(page_number):
    url = f'{base_url}&page={page_number}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    titles = soup.find_all('td', attrs={"style": "font-size:12pt;"})
    publishers_developers = soup.find_all('td', attrs={"width": "100"})
    data_rows = soup.find_all('td', attrs={"align": "center"})
    
    dev_pub = [x.text.strip() for x in publishers_developers]
    data_group = [x for x in data_rows]
    
    for title in titles:
        title_data.append(title.text.replace("Read the review", "").strip())
    
    for i in range(0, len(dev_pub), 2):
        publisher_data.append(dev_pub[i])
    for i in range(1, len(dev_pub), 2):
        developer_data.append(dev_pub[i])
    
    for i in range(1, len(data_group), 12):
        img_tag = data_group[i-1].find('img')
        if img_tag is not None:
            console_data.append(img_tag.get('alt', ''))
        else:
            console_data.append('')  # O algún valor predeterminado para indicar la falta de datos

        vgchartz_score_data.append(data_group[i].text.strip())
        critic_score_data.append(data_group[i+1].text.strip())
        user_score_data.append(data_group[i+2].text.strip())
        
        # Check length before accessing to avoid IndexError
        if len(data_group) > i + 8:
            total_shipped_data.append(data_group[i+3].text.strip())
            total_sales_data.append(data_group[i+4].text.strip())
            na_sales_data.append(data_group[i+5].text.strip())
            pal_sales_data.append(data_group[i+6].text.strip())
            japan_sales_data.append(data_group[i+7].text.strip())
            other_sales_data.append(data_group[i+8].text.strip())
            release_data.append(data_group[i+9].text.strip())
            last_update_data.append(data_group[i+10].text.strip())
        else:
            # Append empty strings to maintain data alignment
            total_shipped_data.append('')
            total_sales_data.append('')
            na_sales_data.append('')
            pal_sales_data.append('')
            japan_sales_data.append('')
            other_sales_data.append('')
            release_data.append('')
            last_update_data.append('')

# Determine the number of pages
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Fix for extracting total results
result_text = soup.find('th', attrs={"colspan": "3"}).text.strip()
result_numbers = ''.join(filter(str.isdigit, result_text))  # Remove non-numeric characters
total_results = int(result_numbers)

results_per_page = 200
total_pages = math.ceil(total_results / results_per_page)

# Scrape each page
for page_number in range(1, total_pages + 1):
    print(f'Scraping page {page_number} of {total_pages}...')
    scrape_page(page_number)
    time.sleep(1)  # Respectful scraping, pause between requests

# Ensure all lists have the same length
max_length = max(len(title_data), len(console_data), len(publisher_data), len(developer_data), 
                 len(vgchartz_score_data), len(critic_score_data), len(user_score_data), 
                 len(total_shipped_data), len(total_sales_data), len(na_sales_data), 
                 len(pal_sales_data), len(japan_sales_data), len(other_sales_data), 
                 len(release_data), len(last_update_data))

def pad_list(lst, target_length):
    while len(lst) < target_length:
        lst.append('')

# Pad all lists to ensure equal length
pad_list(title_data, max_length)
pad_list(console_data, max_length)
pad_list(publisher_data, max_length)
pad_list(developer_data, max_length)
pad_list(vgchartz_score_data, max_length)
pad_list(critic_score_data, max_length)
pad_list(user_score_data, max_length)
pad_list(total_shipped_data, max_length)
pad_list(total_sales_data, max_length)
pad_list(na_sales_data, max_length)
pad_list(pal_sales_data, max_length)
pad_list(japan_sales_data, max_length)
pad_list(other_sales_data, max_length)
pad_list(release_data, max_length)
pad_list(last_update_data, max_length)

# Create DataFrame
df = pd.DataFrame({
    'Title': title_data,
    'Console': console_data,
    'Publisher': publisher_data,
    'Developer': developer_data,
    'VGChartz Score': vgchartz_score_data,
    'Critic Score': critic_score_data,
    'User Score': user_score_data,
    'Total Shipped': total_shipped_data,
    'Total Sales': total_sales_data,
    'NA Sales': na_sales_data,
    'PAL Sales': pal_sales_data,
    'Japan Sales': japan_sales_data,
    'Other Sales': other_sales_data,
    'Release Date': release_data,
    'Last Update': last_update_data
})

# Save DataFrame to CSV
df.to_csv('vg_data1.csv', index=False)

print('Scraping completed and data saved to vg_data1.csv')


Scraping page 1 of 328...
Scraping page 2 of 328...
Scraping page 3 of 328...
Scraping page 4 of 328...
Scraping page 5 of 328...
Scraping page 6 of 328...
Scraping page 7 of 328...
Scraping page 8 of 328...
Scraping page 9 of 328...
Scraping page 10 of 328...
Scraping page 11 of 328...
Scraping page 12 of 328...
Scraping page 13 of 328...
Scraping page 14 of 328...
Scraping page 15 of 328...
Scraping page 16 of 328...
Scraping page 17 of 328...
Scraping page 18 of 328...
Scraping page 19 of 328...
Scraping page 20 of 328...
Scraping page 21 of 328...
Scraping page 22 of 328...
Scraping page 23 of 328...
Scraping page 24 of 328...
Scraping page 25 of 328...
Scraping page 26 of 328...
Scraping page 27 of 328...
Scraping page 28 of 328...
Scraping page 29 of 328...
Scraping page 30 of 328...
Scraping page 31 of 328...
Scraping page 32 of 328...
Scraping page 33 of 328...
Scraping page 34 of 328...
Scraping page 35 of 328...
Scraping page 36 of 328...
Scraping page 37 of 328...
Scraping p

In [341]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

# URL base
base_url = 'https://www.vgchartz.com/games/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=200&order=Sales&showtotalsales=0&showtotalsales=1&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&shownasales=1&showdeveloper=0&showdeveloper=1&showcriticscore=0&showpalsales=0&showpalsales=1&showreleasedate=0&showreleasedate=1&showuserscore=0&showjapansales=0&showjapansales=1&showlastupdate=0&showlastupdate=1&showothersales=0&showothersales=1&showshipped=0&showshipped=1'

# Initialize lists to store data
title_data = []
console_data = []
publisher_data = []
developer_data = []
vgchartz_score_data = []
critic_score_data = []
user_score_data = []
total_shipped_data = []
total_sales_data = []
na_sales_data = []
pal_sales_data = []
japan_sales_data = []
other_sales_data = []
release_data = []
last_update_data = []

# Scraping function
def scrape_page(page_number):
    url = f'{base_url}&page={page_number}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all the rows in the main table
    rows = soup.select('table.chart tr')

    for row in rows[3:]:  # Skip the header rows
        columns = row.find_all('td')
        
        if len(columns) < 15:
            continue  # Skip rows that don't have enough columns

        # Extract data based on known positions in the row
        title_data.append(columns[2].text.strip())
        console_data.append(columns[3].find('img')['alt'] if columns[3].find('img') else '')
        publisher_data.append(columns[4].text.strip())
        developer_data.append(columns[5].text.strip())
        vgchartz_score_data.append(columns[6].text.strip())
        critic_score_data.append(columns[7].text.strip())
        user_score_data.append(columns[8].text.strip())
        total_shipped_data.append(columns[9].text.strip())
        total_sales_data.append(columns[10].text.strip())
        na_sales_data.append(columns[11].text.strip())
        pal_sales_data.append(columns[12].text.strip())
        japan_sales_data.append(columns[13].text.strip())
        other_sales_data.append(columns[14].text.strip())
        release_data.append(columns[15].text.strip())
        last_update_data.append(columns[16].text.strip())

# Determine the number of pages
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

result_text = soup.find('th', attrs={"colspan": "3"}).text.strip()
result_numbers = ''.join(filter(str.isdigit, result_text))  # Remove non-numeric characters
total_results = int(result_numbers)

results_per_page = 200
total_pages = math.ceil(total_results / results_per_page)

# Scrape each page
for page_number in range(1, total_pages + 1):
    print(f'Scraping page {page_number} of {total_pages}...')
    scrape_page(page_number)
    time.sleep(1)  # Respectful scraping, pause between requests

# Ensure all lists have the same length
max_length = max(len(title_data), len(console_data), len(publisher_data), len(developer_data), 
                 len(vgchartz_score_data), len(critic_score_data), len(user_score_data), 
                 len(total_shipped_data), len(total_sales_data), len(na_sales_data), 
                 len(pal_sales_data), len(japan_sales_data), len(other_sales_data), 
                 len(release_data), len(last_update_data))

def pad_list(lst, target_length):
    while len(lst) < target_length:
        lst.append('')

# Pad all lists to ensure equal length
pad_list(title_data, max_length)
pad_list(console_data, max_length)
pad_list(publisher_data, max_length)
pad_list(developer_data, max_length)
pad_list(vgchartz_score_data, max_length)
pad_list(critic_score_data, max_length)
pad_list(user_score_data, max_length)
pad_list(total_shipped_data, max_length)
pad_list(total_sales_data, max_length)
pad_list(na_sales_data, max_length)
pad_list(pal_sales_data, max_length)
pad_list(japan_sales_data, max_length)
pad_list(other_sales_data, max_length)
pad_list(release_data, max_length)
pad_list(last_update_data, max_length)

# Create DataFrame
df = pd.DataFrame({
    'Title': title_data,
    'Console': console_data,
    'Publisher': publisher_data,
    'Developer': developer_data,
    'VGChartz Score': vgchartz_score_data,
    'Critic Score': critic_score_data,
    'User Score': user_score_data,
    'Total Shipped': total_shipped_data,
    'Total Sales': total_sales_data,
    'NA Sales': na_sales_data,
    'PAL Sales': pal_sales_data,
    'Japan Sales': japan_sales_data,
    'Other Sales': other_sales_data,
    'Release Date': release_data,
    'Last Update': last_update_data
})

# Save DataFrame to CSV
df.to_csv('vg_data2.csv', index=False)

print('Scraping completed and data saved to vg_data2.csv')


Scraping page 1 of 328...
Scraping page 2 of 328...
Scraping page 3 of 328...
Scraping page 4 of 328...
Scraping page 5 of 328...
Scraping page 6 of 328...
Scraping page 7 of 328...
Scraping page 8 of 328...
Scraping page 9 of 328...
Scraping page 10 of 328...
Scraping page 11 of 328...
Scraping page 12 of 328...
Scraping page 13 of 328...
Scraping page 14 of 328...
Scraping page 15 of 328...
Scraping page 16 of 328...
Scraping page 17 of 328...
Scraping page 18 of 328...
Scraping page 19 of 328...
Scraping page 20 of 328...
Scraping page 21 of 328...
Scraping page 22 of 328...
Scraping page 23 of 328...
Scraping page 24 of 328...
Scraping page 25 of 328...
Scraping page 26 of 328...
Scraping page 27 of 328...
Scraping page 28 of 328...
Scraping page 29 of 328...
Scraping page 30 of 328...
Scraping page 31 of 328...
Scraping page 32 of 328...
Scraping page 33 of 328...
Scraping page 34 of 328...
Scraping page 35 of 328...
Scraping page 36 of 328...
Scraping page 37 of 328...
Scraping p

In [346]:
total_sales_data

['N/A',
 '28th Sep 98',
 '',
 'N/A',
 'N/A',
 'N/A',
 '08th Oct 20',
 '200.00m',
 'N/A',
 'N/A',
 '18th Dec 87',
 '',
 'N/A',
 'N/A',
 'N/A',
 '27th Feb 20',
 '129.15m',
 'N/A',
 'N/A',
 '21st Sep 04',
 '',
 'N/A',
 'N/A',
 'N/A',
 '05th Feb 20',
 '82.90m',
 'N/A',
 'N/A',
 '17th Nov 09',
 '',
 'N/A',
 'N/A',
 'N/A',
 '13th Feb 20',
 '73.49m',
 'N/A',
 'N/A',
 '15th Nov 01',
 '',
 'N/A',
 'N/A',
 'N/A',
 '19th Nov 18',
 '60.00m',
 'N/A',
 'N/A',
 '16th May 11',
 '',
 'N/A',
 'N/A',
 'N/A',
 '11th Feb 20',
 '51.00m',
 'N/A',
 'N/A',
 '24th Apr 12',
 '',
 'N/A',
 'N/A',
 'N/A',
 '07th Feb 20',
 '43.80m',
 'N/A',
 'N/A',
 '07th Nov 06',
 '',
 'N/A',
 'N/A',
 'N/A',
 '17th Feb 20',
 '37.38m',
 'N/A',
 'N/A',
 '21st Aug 98',
 '',
 'N/A',
 'N/A',
 'N/A',
 '02nd Oct 22',
 '33.71m',
 'N/A',
 'N/A',
 '10th May 10',
 '',
 'N/A',
 'N/A',
 'N/A',
 '03rd Feb 20',
 '31.38m',
 'N/A',
 'N/A',
 '17th Nov 02',
 '',
 'N/A',
 'N/A',
 'N/A',
 '06th Feb 20',
 '30.00m',
 'N/A',
 'N/A',
 '26th Sep 95',
 '',
 

In [342]:
df_vg_data = pd.read_csv('/Users/Jaimecalderon/Desktop/IRONHACK/Proyectos/Proyecto final/Videojuegos/vg_data2.csv')

/var/folders/zc/6rc7nzhj13scy5qssxw2pjlh0000gn/T/ipykernel_7279/874623303.py:1: DtypeWarning: Columns (1,4,5,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vg_data = pd.read_csv('/Users/Jaimecalderon/Desktop/IRONHACK/Proyectos/Proyecto final/Videojuegos/vg_data2.csv')


In [343]:
df_vg_data

,Title,Console,Publisher,Developer,VGChartz Score,Critic Score,User Score,Total Shipped,Total Sales,NA Sales,PAL Sales,Japan Sales,Other Sales,Release Date,Last Update
0,Tetris,NaN,The Tetris Company,Alexey Pajitnov,520.00m,NaN,NaN,NaN,NaN,NaN,01st Jan 88,27th Feb 20,NaN,483.85m,NaN
1,Pokemon,NaN,Nintendo,Game Freak,NaN,NaN,NaN,NaN,28th Sep 98,03rd Feb 20,NaN,430.00m,NaN,NaN,NaN
2,Grand Theft Auto,NaN,Rockstar Games,Rockstar North,NaN,NaN,27th Mar 98,03rd Feb 20,NaN,425.00m,NaN,NaN,NaN,NaN,NaN
3,Call of Duty,NaN,Activision,Infinity Ward,29th Oct 03,03rd Feb 20,NaN,421.12m,NaN,NaN,NaN,NaN,NaN,20th Jul 83,20th Feb 20
4,Super Mario,20th Feb 20,Nintendo,Nintendo,NaN,325.00m,NaN,NaN,NaN,NaN,NaN,15th Dec 93,03rd Feb 20,NaN,300.00m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65572,Zombillie,NaN,Forever Entertainment S.A.,Forever Entertainment S.A.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65573,Zone of the Enders: The 2nd Runner MARS,NaN,Konami,Cygames,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65574,Zoo Tycoon: Ultimate Animal Collection,NaN,Microsoft Studios,Frontier Developments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65575,Zoo Tycoon: Ultimate Animal Collection,NaN,THQ Nordic,Frontier Developments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_15_24_vg = df_vg_data[(df_vg_data['year'] >= 2015) & (df_prueba['year'] <= 2024)]

In [347]:
from bs4 import BeautifulSoup as bs
url = 'https://www.vgchartz.com/games/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=200&order=Sales&showtotalsales=0&showtotalsales=1&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&shownasales=1&showdeveloper=0&showdeveloper=1&showcriticscore=0&showpalsales=0&showpalsales=1&showreleasedate=0&showreleasedate=1&showuserscore=0&showjapansales=0&showjapansales=1&showlastupdate=0&showlastupdate=1&showothersales=0&showothersales=1&showshipped=0&showshipped=1'
response = requests.get(url)
response

response.content
response.headers

print(response.headers['Content-Type'])

#PARA ACCEDER AL CONTENIDO DE MANERA MAS LEGIBLE
soup = bs(response.content, 'html.parser')

text/html; charset=UTF-8


In [351]:
soup.find_all('th')

[<th align="left" colspan="3" style="color:white;">Results: (65,577)</th>,
 <th align="right" colspan="11" style="text-align:right;"><span><a href="/games/games.php?page=1&amp;results=200&amp;order=Sales&amp;ownership=Both&amp;direction=DESC&amp;showtotalsales=1&amp;shownasales=1&amp;showpalsales=1&amp;showjapansales=1&amp;showothersales=1&amp;showpublisher=1&amp;showdeveloper=1&amp;showreleasedate=1&amp;showlastupdate=1&amp;showvgchartzscore=0&amp;showcriticscore=0&amp;showuserscore=0&amp;showshipped=1"> &lt;&lt; </a><a class="selected" href="/games/games.php?page=1&amp;results=200&amp;order=Sales&amp;ownership=Both&amp;direction=DESC&amp;showtotalsales=1&amp;shownasales=1&amp;showpalsales=1&amp;showjapansales=1&amp;showothersales=1&amp;showpublisher=1&amp;showdeveloper=1&amp;showreleasedate=1&amp;showlastupdate=1&amp;showvgchartzscore=0&amp;showcriticscore=0&amp;showuserscore=0&amp;showshipped=1"> 1 </a></span><span><a href="/games/games.php?page=2&amp;results=200&amp;order=Sales&amp